In [ ]:
!pip install tensorflow

In [1]:

from tensorflow import keras
from tensorflow.keras import layers, models, callbacks



from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import mean_absolute_error

from tensorflow.keras.callbacks import EarlyStopping

In [2]:
def string_number(x):
    x = x.lower()
    new = ""
    for each in x :
        new = new + str( ord(each) )
    return new


def char_number(x):    
    return str( ord(x)  )



def getConvertedFormatofText(x):

    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(x.values.astype('U'))
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

    return tfidf_transformer


def getValidationData(xtest, ytest):
    xtest, xvalid, ytest, yvalid = train_test_split(xtest, ytest, test_size=0.50, random_state=42)
    return xtest, xvalid, ytest, yvalid


def getWordSplitData():
    data = pd.read_csv("word_split_replacement_encoded.tsv", sep="\t")
    data.columns = ["words", "output", "wordlen"]
    #     data = data.dropna()    
    x = data.words.map(char_number)
    x = x.values.astype('int32')
    y = data.output
    y = y.values.astype('int64')
    return y, x


def getWordSplitOHEData():
    data = pd.read_csv("word_splits_ohe.tsv", sep="\t")
#     data = data.dropna()
    l = [x for x in range(2, data.shape[1])]
    x = data.iloc[:,l]
    x = x.values.astype('int16')
    y = data.output
    y = y.values.astype('int64')
    return x, y


def getTrainTestData():        
    x, y = getWordSplitData()
    xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.40, random_state = 0)
    xtest, xvalid, ytest, yvalid = getValidationData(xtest, ytest)
    return  xtrain, xtest, xvalid, ytrain, ytest, yvalid


In [5]:

def getTheNetwork(num=6):
    i = 0
    list_dense = []    
    list_dense.append(layers.Dense(units= 350 * (num) , activation='relu', input_dim=1))
#     list_dense.append(layers.Dropout(0.3))
    for x in range(2,num):
        i = i+1
        list_dense.append(layers.Dense(units= 350 * ((num)-i), activation='relu' ))        
#         list_dense.append(layers.Dropout(0.3))
        
    
    list_dense.append(layers.Dense(units= 350, activation='relu' ))
    list_dense.append(layers.Dense(units=1))

    return  list_dense

In [6]:
model = keras.Sequential(getTheNetwork(6))

In [7]:


model.compile(optimizer='adam', loss = 'huber', metrics=['mse', 'mae', 'mape'])


early_stopping = EarlyStopping(
    min_delta=0.01,
    patience=40,
    restore_best_weights=True
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2100)              4200      
_________________________________________________________________
dense_1 (Dense)              (None, 1750)              3676750   
_________________________________________________________________
dense_2 (Dense)              (None, 1400)              2451400   
_________________________________________________________________
dense_3 (Dense)              (None, 1050)              1471050   
_________________________________________________________________
dense_4 (Dense)              (None, 700)               735700    
_________________________________________________________________
dense_5 (Dense)              (None, 350)               245350    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 3

In [4]:
xtrain, xtest, xvalid, ytrain, ytest, yvalid = getTrainTestData()

In [9]:
export_path_keras = "replacement_file_hacker_350.h5"
save_checkpoint = keras.callbacks.ModelCheckpoint(
    export_path_keras, monitor='loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='min', save_freq='epoch'
)

In [10]:
early_stopping = EarlyStopping(
    min_delta=0.01,
    patience=40,
    restore_best_weights=True
)

In [ ]:

history = model.fit(
    xtrain, ytrain,
    validation_data=(xvalid, yvalid),
    batch_size=100,
    epochs=20,
    callbacks=[early_stopping, save_checkpoint],
    verbose=1
)


In [3]:
model = models.load_model('replacement_file_hacker_350.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2100)              4200      
_________________________________________________________________
dense_1 (Dense)              (None, 1750)              3676750   
_________________________________________________________________
dense_2 (Dense)              (None, 1400)              2451400   
_________________________________________________________________
dense_3 (Dense)              (None, 1050)              1471050   
_________________________________________________________________
dense_4 (Dense)              (None, 700)               735700    
_________________________________________________________________
dense_5 (Dense)              (None, 350)               245350    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 3

In [12]:
ypred = model.predict(xtest)

s1 = pd.Series(ypred.flatten())
ypred1 = s1.map(floor)

print(mean_absolute_error(ytest, ypred1)) 
print(mean_absolute_error(ytest, ypred))

1.1696453647203002
1.282789699781765


(None, None)

In [13]:
print(mean_absolute_error(ytest, ypred1)) 
print(mean_absolute_error(ytest, ypred))

1.1696453647203002
1.282789699781765


In [17]:
ytest - ypred1

0         0
1         1
2         0
3         0
4         0
         ..
626667    0
626668    0
626669    0
626670    0
626671    0
Length: 626672, dtype: int64

In [19]:
ytest - ypred.flatten()

array([-0.93352509,  0.0932312 , -0.15602112, ..., -0.93352509,
       -0.23227692, -0.35005188])

In [6]:
xtrain[0+5:5+5], ytrain[0+5:5+5]

(array([3240, 2130, 3333, 1111, 4110]),
 array([ 99, 110,  50,  48, 112], dtype=int32))

In [23]:
def num_char(x):    
    return str( chr(x)  )

In [29]:
from math import ceil, floor


ypred = model.predict([[1980],[5220],[1260],[1890],[3240],[8190]])

s1 = pd.Series(ypred.flatten())
print(ypred)
s1 = s1.map(floor)
print(s1)
s1 = s1.map(num_char)
s1

[[115.19685 ]
 [101.933525]
 [ 97.02582 ]
 [114.19497 ]
 [ 99.24246 ]
 [107.3463  ]]
0    115
1    101
2     97
3    114
4     99
5    107
dtype: int64


0    s
1    e
2    a
3    r
4    c
5    k
dtype: object

In [ ]:

history_df = pd.DataFrame(history.history)
history_df.loc[5:, ['loss', 'val_loss']].plot()
history_df.loc[5:, ['mae', 'val_mae']].plot()

print(history_df['val_loss'].min())
print(history_df['val_mae'].min())

In [3]:
#convert a word to single char cell

def writeRow(char, enc, wlen):
     with open("word_split_replacement_encoded.tsv", 'a') as decodeMsgWritter:
        decodeMsgWritter.write("\n{0}\t{1}\t{2}".format(char,enc,wlen))


def convertDatato_chars_encryption(x):
    row_x = x.words; row_y = x.output;
#     row_x = list(row_x.lower())
    for each in range(4,len(row_y)+4,4):
        if each-4 == 0:
            writeRow(row_x[int(each/4)-1],row_y[each-4:each], len(row_x) )            
        else:
            writeRow(row_x[int(each/4)-1],row_y[each-4:each], 0 )

    
    
    
data = pd.read_csv("title_encoded.tsv", sep="\t")
data.columns=["words","output"]
data = data.dropna()


data.apply(convertDatato_chars_encryption, axis = 1)
print('completed')

completed


In [17]:
data = pd.read_csv("word_split_replacement_encoded.tsv", sep="\t")
data.columns=["words","output", "wordlen"]

data = pd.concat([data ,pd.get_dummies(data['words'], prefix='char')],axis=1)
data.drop(['words'], axis=1, inplace=True)

In [18]:
data.head()

,output,wordlen,char_0,char_1,char_2,char_3,char_4,char_5,char_6,char_7,...,char_q,char_r,char_s,char_t,char_u,char_v,char_w,char_x,char_y,char_z
0,5220,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2430,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2070,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,5120,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2520,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
data.to_csv("word_splits_ohe.tsv", sep="\t", index=False)